In [1]:
%pylab
import sys
sys.path.append('..')
import uit_scripts.shotnoise.gen_shot_noise as gsn
import uit_scripts.stat_analysis.deconv_methods as dec

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
dt = 1e-2

seednum = 0

if seednum == 0: # finding pulse shape converges
    gamma = 0.5
    K = int(1e2)
    seed = 1234
elif seednum== 1: # finding pulse shape diverges
    gamma = 0.5
    K = int(1e2)
    seed = 333
elif seednum == 2: # finding pulse shape diverges
    gamma = 3.
    K = int(1e4)
    seed = 12345
    
T, S, A, ta = gsn.make_signal(gamma, K, dt, ampta=True, seedTW=seed, seedA=seed)

kern = gsn.kern(T-T.mean())

ta_index = np.ceil(ta/dt).astype(int)
F = np.zeros(T.size)
for i in range(ta_index.size):
    F[ta_index[i]] += A[i]  # This may need to be multiplied by td.

plt.close('data')
plt.figure('data')
plt.plot(T, S, label='Phi_K')
plt.plot(T, F, '-o', label='f_K')
plt.plot(T, kern, label='phi')
plt.xlabel('t/td')
#plt.ylabel('Phi')
plt.legend()

## Estimate kernel function from data

In [6]:
res, err =  dec.RL_gauss_deconvolve(S, F, 200)

plt.close('err')
plt.figure('err')
plt.loglog(err[1:])

plt.close('res')
plt.figure('res')
plt.plot(T-T.mean(), kern, label='kern')
plt.plot(T-T.mean(), res, ':', label='res')
plt.legend()
plt.xlabel('t/td')

i = 200
Minimum error at iteration i = 65


Text(0.5, 0, 't/td')

## Estimate pulse arrivals from data

In [4]:
res, err = dec.RL_gauss_deconvolve(S, kern, 200)
res = res[:,0]

plt.close('err')
plt.figure('err')
plt.loglog(err[1:])

plt.close('res')
plt.figure('res')
plt.plot(T, F, '-o', label='f_K')
plt.plot(T, res, label='res')
plt.legend()
plt.xlabel('t/td')

ta_est, amp_est = dec.find_amp_ta_savgol(res, T)

plt.close('ampta')
plt.figure('ampta')
plt.plot(ta, A, 'o', label='true')
plt.plot(ta_est, amp_est, 'd', label='est')
plt.legend()
plt.xlabel('t/td')
plt.ylabel('A')

i = 200
Minimum error at iteration i = 200


Text(0, 0.5, 'A')